# Étape 1 : Préparer l’environnement

In [150]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os

Créez une session Spark pour démarrer PySpark :

In [151]:
# Créez une session Spark
spark = SparkSession.builder.appName("Manipulez des images avec PySpark") \
    .master("local[2]") \
    .getOrCreate()

# Étape 2 : Charger des images de chats et de chiens

Le dataset Dogs vs Cats est disponible sur Kaggle : [Dogs vs Cats Dataset](https://www.kaggle.com/c/dogs-vs-cats/data).

Téléchargez et extrayez les images dans un dossier (par exemple, cats_dogs/).

**N'hésitez pas à réduire la taille du jeu de données** dans un premier temps. Quand on cuisine un code, on a pas besoin de tout le dataset ! Ce n'est qu'une fois que le code est bon qu'on test avec le dataset dans son ensemble. Dans mon cas, je n'ai conservé que 21 images par classe dans mon fichier cats_dogs_mini.zip.

Puisque j'ai uploadé un fichier zippé, je dois d'abord le dézipper.

In [152]:
%%capture
# !unzip cats_dogs_mini.zip

In [153]:
# Définissez le chemin vers votre dataset (à adapter selon votre structure de fichiers)
DATASET_PATH = "cats_dogs_mini"

Cette fonction charge une image, la redimensionne (pour correspondre à l’entrée du CNN), et la prétraite.

In [154]:
# Chargez les chemins des fichiers et les classes (cat/dog)
def extract_label(filepath):
    return "cat" if "cat" in filepath.split("/")[-1] else "dog"

def get_image_paths(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
      for filename in files:
        if filename.endswith(".jpg"):
            image_paths.append(os.path.join(root, filename))
    return image_paths

## Calcul distribué :

Cette partie du code est prit en charge automtiquement par PySpark pour être distribuée si possible. En effet, le DataFrame Spark est réparti entre les nœuds du cluster.

Chaque nœud traite un sous-ensemble des données (par exemple, un lot d'images).

Chargez les images et leurs étiquettes ("chat" ou "chien") dans un DataFrame :

In [155]:
# Liste des chemins
image_paths = get_image_paths(DATASET_PATH)

# Créer un DataFrame Spark à partir des chemins
image_df = spark.createDataFrame([(path, extract_label(path)) for path in image_paths], ["path", "label"])

In [156]:
# Affichez quelques lignes du DataFrame
image_df.show(5, truncate=False)

+--------------------------------+-----+
|path                            |label|
+--------------------------------+-----+
|cats_dogs_mini/dogs/dog.4252.jpg|dog  |
|cats_dogs_mini/dogs/dog.4258.jpg|dog  |
|cats_dogs_mini/dogs/dog.4269.jpg|dog  |
|cats_dogs_mini/dogs/dog.4253.jpg|dog  |
|cats_dogs_mini/dogs/dog.4265.jpg|dog  |
+--------------------------------+-----+
only showing top 5 rows



## Calcul distribué :
Les opérations comme `groupBy` sont distribuées, chaque partition effectuant un traitement local avant agrégation globale.

In [157]:
# Comptez le nombre d'images par classe
image_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  dog|   21|
|  cat|   21|
+-----+-----+



# Étape 3 : Catégorisation automatique

In [158]:
# Réalisez une classification simple avec les caractéristiques extraites.
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [159]:
# Charger le modèle pré-entrainé
model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

/tmp/ipykernel_335639/4091702483.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


In [160]:
# Fonction pour extraire les caractéristiques d'une image
def extract_features(path):
    img = load_img(path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    features = model.predict(img_array)
    return Vectors.dense(features.flatten())

## Calcul distribué :
L'UDF `extract_features` est appliquée en parallèle sur chaque partition des données.

In [161]:
# Déclarez la fonction utilisateur (UDF)
from pyspark.sql.types import ArrayType, FloatType
# feature_udf = udf(extract_features, ArrayType(FloatType()))
feature_udf = udf(extract_features, VectorUDT())

In [162]:
# Ajoutez une colonne avec les caractéristiques extraites
image_df = image_df.withColumn("features", feature_udf(col("path")))

In [163]:
# Affichez le DataFrame avec les nouvelles colonnes
image_df.show(5)

2025-01-20 21:26:20.511027: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 21:26:20.515400: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 21:26:20.523987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737408380.538943  543984 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737408380.543259  543984 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 21:26:20.558896: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

+--------------------+-----+--------------------+
|                path|label|            features|
+--------------------+-----+--------------------+
|cats_dogs_mini/do...|  dog|[0.26830497384071...|
|cats_dogs_mini/do...|  dog|[1.00685632228851...|
|cats_dogs_mini/do...|  dog|[0.05627692490816...|
|cats_dogs_mini/do...|  dog|[0.0,0.0,1.145988...|
|cats_dogs_mini/do...|  dog|[0.06609643995761...|
+--------------------+-----+--------------------+
only showing top 5 rows



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [164]:
# Indexer les labels pour la classification
from pyspark.ml.feature import StringIndexer
label_indexer = StringIndexer(inputCol="label", outputCol="label_index")
# data = label_indexer.fit(vectorized_df).transform(vectorized_df)
data = label_indexer.fit(image_df).transform(image_df)

In [165]:
data.show(5)

2025-01-20 21:26:25.353901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 21:26:25.356778: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 21:26:25.365223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737408385.380149  544264 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737408385.384454  544264 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 21:26:25.399652: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

+--------------------+-----+--------------------+-----------+
|                path|label|            features|label_index|
+--------------------+-----+--------------------+-----------+
|cats_dogs_mini/do...|  dog|[0.26830497384071...|        1.0|
|cats_dogs_mini/do...|  dog|[1.00685632228851...|        1.0|
|cats_dogs_mini/do...|  dog|[0.05627692490816...|        1.0|
|cats_dogs_mini/do...|  dog|[0.0,0.0,1.145988...|        1.0|
|cats_dogs_mini/do...|  dog|[0.06609643995761...|        1.0|
+--------------------+-----+--------------------+-----------+
only showing top 5 rows



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [166]:
# Diviser les données en ensembles d'entraînement et de test
train, test = data.randomSplit([0.8, 0.2], seed=42)

## Calcul distribué :
Spark ML divise les données d'entraînement entre les nœuds pour entraîner le modèle de manière parallèle.

In [167]:
%%time
# Entraîner un modèle de régression logistique
lr = LogisticRegression(featuresCol="features", labelCol="label_index")
model = lr.fit(train)

2025-01-20 21:26:31.230207: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 21:26:31.233220: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 21:26:31.241954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737408391.258143  545024 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737408391.262390  545024 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 21:26:31.266384: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be

CPU times: user 51.7 ms, sys: 0 ns, total: 51.7 ms
Wall time: 11 s


In [168]:
%%time
# Évaluer le modèle sur les données de test
predictions = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Précision du modèle : {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 887ms/step 2) / 2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 929ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/

Précision du modèle : 1.0
CPU times: user 21.7 ms, sys: 4.17 ms, total: 25.9 ms
Wall time: 3.68 s


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step+ 1) / 2]


In [170]:
# Arrêt de la session Spark
spark.stop()